In [1]:
import scanpy as sc
import muon as mu
import numpy as np
import pandas as pd
import mofax as mofa
import seaborn as sns
import matplotlib.pyplot as plt
import pyro
from pyro.nn import PyroSample, PyroModule
from pyro.infer import SVI, Trace_ELBO, autoguide
import torch
import torch.nn.functional as F
from torch.nn.functional import softplus
from sklearn.metrics import mean_squared_error
import random
import seaborn as sns
import muon as mu
import anndata

/scratch/venvs/deeplife2/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dir="/scratch/deeplife/"
pbmc = sc.read_10x_h5(dir+"5k_pbmc_protein_v3_nextgem_filtered_feature_bc_matrix.h5", gex_only=False)
pbmc.var_names_make_unique()
pbmc.layers["counts"] = pbmc.X.copy()

/scratch/venvs/deeplife2/lib/python3.12/site-packages/anndata/_core/anndata.py:1820: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [3]:
protein = pbmc[:, pbmc.var["feature_types"] == "Antibody Capture"].copy()
rna = pbmc[:, pbmc.var["feature_types"] == "Gene Expression"].copy()
# protein.X = protein.X.toarray()
# rna.X = rna.X.toarray()
# for k in rna.layers:
#     rna.layers[k] = rna.layers[k].toarray()
# for k in protein.layers:
#     protein.layers[k] = protein.layers[k].toarray()


In [4]:
#neur_ips = anndata.read_h5ad(dir+"neurips2021.h5ad")

In [5]:
mudata = mu.MuData({
    "protein": protein,
    "rna": rna
})

In [8]:
mu.tl.mofa(mudata, n_factors=3, convergence_mode='medium', outfile="models/muon_mofa.hdf5", gpu_mode=True, use_obs="union", use_float32=True, likelihoods="gaussian")


        #########################################################
        ###           __  __  ____  ______                    ### 
        ###          |  \/  |/ __ \|  ____/\    _             ### 
        ###          | \  / | |  | | |__ /  \ _| |_           ### 
        ###          | |\/| | |  | |  __/ /\ \_   _|          ###
        ###          | |  | | |__| | | / ____ \|_|            ###
        ###          |_|  |_|\____/|_|/_/    \_\              ###
        ###                                                   ### 
        ######################################################### 
       
 
        
use_float32 set to True: replacing float64 arrays by float32 arrays to speed up computations...

Loaded view='protein' group='group1' with N=5527 samples and D=32 features...
Loaded view='rna' group='group1' with N=5527 samples and D=33538 features...



Model options:
- Automatic Relevance Determination prior on the factors: True
- Automatic Relevance Determination prior on the

In [3]:
import h5py
f=h5py.File("models/muon_mofa.hdf5", "r")

In [10]:
print(f.keys())

f["data"]

<KeysViewHDF5 ['data', 'expectations', 'features', 'features_metadata', 'groups', 'intercepts', 'model_options', 'samples', 'training_opts', 'training_stats', 'variance_explained', 'views']>


<KeysViewHDF5 ['group1']>